<a href="https://colab.research.google.com/github/Subhasishbasak/NLP/blob/master/Probability_language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TA_Assignment_4/MDS201803

In [0]:
import re
import pickle
import random
from nltk.corpus import reuters
from nltk import bigrams, trigrams
from collections import Counter, defaultdict

In [0]:
import nltk
nltk.download('reuters')
nltk.download('punkt')

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
f = open('/content/drive/My Drive/CorpusFileName.txt')
raw = f.read()

Preprcessing of the data

In this section we remove some unnecessary symbols (for e.g \n, =, → etc.). Since we dont want this symbols to come when we are predicting the next most probable word using our language model.

In [0]:
raw = re.sub("\n"," ",raw)         # removing newline command
raw = re.sub("="," ",raw)          # removing '=' symbol
raw = re.sub("→"," ",raw)          # removing '→' symbol        
raw = re.sub("[#$%&\'()*+-/<=>?@[\\]^_`{|}~\"]","",raw)  # symbols not commonly used in bengali literature and hence we remove them    

Extracting sentences

In [0]:
# This code takes the raw preprocessed corpus and breakes it into sentences.
# Each sentence is read as a list of words in "sentence_corpus" list

sentence_corpus = []
sentence = str()
for i in raw:
  if i == '।':
    temp = sentence.split(' ')
    if '' in temp:
      temp.remove('')       # removing whitespace
    sentence_corpus.append(temp)
    sentence = str()
  else:
    sentence = sentence + i  

Creating the 3-gram language model

In [0]:
# Creating a dictionary for model
model = defaultdict(lambda: defaultdict(lambda: 0))

# This code creates the trigrams from the sentences and creates a dictionary with the first 2 tokens as keys and the target token as value
for sentence in sentence_corpus:
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
          model[(w1, w2)][w3] += 1
 
# This code transforms the counts into relative frequencies (probabilities)
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

To find the most probable starting words of a Bengali sentence

In [0]:
# This dictionary stores all the unique starting words of the sentences and their frequencies. This helps to identify the most probable starting words.
first_word = {}
for i in sentence_corpus:
    if i != [] and i[0] != '':
      try:
        first_word[i[0]] += 1
      except KeyError:
        first_word[i[0]] = 1

# We sort the dictionary w.r.t their frequencies
sorted_x = sorted(first_word.items(), key=lambda kv: kv[1], reverse=True)

In [0]:
# The top 10 most probable starting words are :
sorted_x[:10]

[('এই', 72236),
 ('তিনি', 47698),
 ('এটি', 33586),
 ('তার', 29004),
 ('এর', 23802),
 ('যদিও', 17318),
 ('১৯৩৭সালে', 17306),
 ('এ', 16559),
 ('উকাইর', 16445),
 ('তাদের', 16345)]

In [0]:
# In our 3-gram model we take the two starting tokens ['এই', 'বাংলা'] and print all the following words along with their probabilities
dict(model['এই', 'ছিল'])

{'অষ্টাদশ': 0.03125,
 'আমাদের': 0.03125,
 'এই': 0.03125,
 'এক': 0.03125,
 'একটি': 0.03125,
 'ক্যাপ্টেন': 0.03125,
 'গাসানীয়': 0.03125,
 'চলচ্চিত্র': 0.03125,
 'তাঁর': 0.03125,
 'তার': 0.125,
 'না': 0.03125,
 'প্রথম': 0.03125,
 'প্রথাগত': 0.03125,
 'বাইজেন্টাইন': 0.03125,
 'মনে': 0.03125,
 'যে': 0.25,
 'রুট': 0.03125,
 'শেষ': 0.03125,
 'শেষকৃত্যের': 0.03125,
 'সংলাপ': 0.03125,
 'সাধারণ': 0.03125,
 'সেই': 0.03125}

Predicting sentences

In [0]:
# The following code is used to generate sentences for a given pair of tokens 
# Here we start with the pair of words ['এই', 'ছিল'], since 'এই' is the most probable starting word for our corpus.

# Given two words the following code extracts the most probable next word using the Tri-gram model and appends it to the sentence. 
# In the next step the last two words of this sentence is considered and the same step is followed to generate the next word.

text = ['এই', 'ছিল']
sentence_finished = False
 
while not sentence_finished:

  max_prob = .0

  for word in model[tuple(text[-2:])].keys():
        if max_prob < model[tuple(text[-2:])][word]:    # Take the next word with maximum probability
            max_prob = model[tuple(text[-2:])][word]    
            most_probable_word = word
  text.append(most_probable_word)

  if text[-2:] == [None, None]:                       
      sentence_finished = True
      text.append('।')                                  # Appends the symbol '।' after sentence reaches its end
 

print ("Generated sentence : \n", ' '.join([t for t in text if t]))

print("\nThe length of the generated sentence is : ", len(' '.join([t for t in text if t]).split(' ')))

Generated sentence : 
 এই ছিল যে রাজনৈতিক ক্ষমতা সাবাহদের অধীনে চলে যায়; বাণিজ্যিক পরিবারেরা বাণিজ্যের উপর খেয়াল রাখত যেখানে হাউজ অব সাবাহ এবং অন্যান্য উল্লেখযোগ্য কুয়েতি পরিবারেরা কুয়েতের দেয়ালের ভেতর গড়ে উঠা শহরগুলোতে নিরাপত্তা প্রদান করত ।

The length of the generated sentence is :  34
